In [1]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [11]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ExpectedCond
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [40]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(2)
        #ACCEDEMOS A 'REPARTO PRINCIPAL >
    
        #driver.execute_script("window.scrollTo(0,0)")
    sleep(3)
        #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()
    sleep(2)
    driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
    sleep(5)
    #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary > div > a > h3 > svg

    lista_actores_peli_i=[]

    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,7):
        
        #sleep(5)
        #accedemos a la informacion del actor/actriz:
        
        driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
        sleep(2)
        
        #seleccionamos desplegable, para acceder a biografia y premios:
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
        sleep(2)
        #acceso a la biografia
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
        sleep(2)
        
        # obtenemos el NOMBRE del actor
        nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
        
        try:
            #obtenemos AÑO
            año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
            año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
        except:
            año='NULL'
        sleep(3)

        try:

            #obtenemos el texto para filtrar informacion 'CONOCIDO POR' Y 'A QUE SE DEDICA':
            xpath_d='//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/div[1]/a/h3'
            pyhton_d= driver.find_element(By.XPATH, xpath_d)
            driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d) #hacemos scroll para acceder al texto que necesitamos:
            sleep(6)
            #Obtenemos el texto:
            driver.find_element("xpath", '//*[@id="mini_bio"]').click()
            texto_actor = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text 
            sleep(3)
            conocido_por=[]
            if re.search('conocido por|conocida por', texto_actor) == None:
                    conocido_por='NULL'
            else:
                patron_a='\.'
                patron_b='conocido por|conocida por' # siempre me va a dar un len()=2, por que en la cadena solo hay un 'conocido por' y este siempre estara en [1]
                patron_c='\(\d{4}'
                patron_d="\), |\) y |\)"
                lista_sep=re.split(patron_b, texto_actor)
                lista_sep_2=re.split(patron_a, lista_sep[1])
                # la estring que nos interesa termina con un '.' simpre va a estar en [0] por que es consecutivo al 'conocido por'
                lista_sep_3=re.split(patron_c, lista_sep_2[0])
                    
                for y in lista_sep_3:
                    lista_sep_3_sin_espacios=(re.sub(patron_d,'', y)).strip() #quitamos espacios y parentesis
                    if lista_sep_3_sin_espacios != '': #quitamos de la lista posibles espacios
                        conocido_por.append(lista_sep_3_sin_espacios)
        except:
            conocido_por='NULL'
        
        sleep(2)
        driver.back()
        sleep(10)
        driver.back()
        sleep(10)
        try:
        #volvemos a pagina anterior, de nuevo vamos al desplegable
        #acceso a los PREMIOS
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
        #//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span
        except:
            #si se sale de la pagina, volvemos a ella:
            driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
            sleep(2)
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a los premios
            
            sleep(2)

   
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la biografia
        sleep(6)
        #creamos ciclo for para sacar las los premios:
        
        try:
            premios=[]
            for i in range(1,6): #solo queremos 5 pelis
                premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                premios.append(premio_limpio)
            sleep(6)
        except:
            premios='NULL'

        sleep(2)
        driver.back()
        sleep(3)

        que_hace=[]
        try:
            try:
                for z in range(1,4):
                    peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                    que_hace.append(peli_hace)
                    sleep(2)
            except:
                    peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                    que_hace.append(peli_hace)
                    sleep(2)
        except:
            que_hace='NULL'

        #sacamos tupla de cada actor
        lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
        tupla_x=tuple(lista_infor_actor)
        lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula

        sleep(6)
        driver.back()
        sleep(3)
        #volvemos a la página principal 

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [4]:
#esto cambia segun la ruta donde este guardado el csv

datos_accion_csv =pd.read_csv(r'C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_Acción_2010-2025.csv') 
datos_drama_cvs = pd.read_csv(r'C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_Comedia_2010-2025.csv')
datos_comedia_cvs = pd.read_csv(r'C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_Drama_2010-2025.csv')


In [5]:
id_accion = datos_accion_csv["ID_IMDB"]
id_drama = datos_drama_cvs["ID_IMDB"]
id_comedia = datos_comedia_cvs["ID_IMDB"]

In [6]:
#convierto en listas

lista_id_accion = id_accion.to_list()
lista_id_comedia = id_comedia.to_list()
lista_id_drama = id_drama.to_list()

In [7]:
#  todos los id en una lista

union_listas =[]
union_listas.extend(lista_id_accion)
union_listas.extend(lista_id_comedia)
union_listas.extend(lista_id_drama)

In [14]:
# cuento numero de elementos en la lista
print(len(union_listas))

12985


In [15]:
# elimino id duplicados duplicados

lista_id_set = set(union_listas)
print(len(lista_id_set))

9037


In [16]:
#lo convierto en lista

lista_id = list(lista_id_set)
print(lista_id)

['tt10238762', 'tt10854954', 'tt11655440', 'tt11344078', 'tt11650106', 'tt15649360', 'tt17066254', 'tt15546592', 'tt13357488', 'tt11388604', 'tt13951662', 'tt10202644', 'tt12701118', 'tt10095582', 'tt0790663', 'tt7861480', 'tt2397597', 'tt11791860', 'tt1068242', 'tt23149770', 'tt23018646', 'tt29865716', 'tt3075398', 'tt11799052', 'tt11155912', 'tt3085500', 'tt10125716', 'tt10262172', 'tt10068916', 'tt11196176', 'tt10049768', 'tt10059236', 'tt10078830', 'tt29569025', 'tt10413092', 'tt10294678', 'tt10333730', 'tt10448728', 'tt10010720', 'tt11522680', 'tt11343676', 'tt12000340', 'tt10044220', 'tt13685732', 'tt2990426', 'tt10006412', 'tt12093528', 'tt10871576', 'tt10268880', 'tt11063978', 'tt10355546', 'tt12038484', 'tt10021758', 'tt0980999', 'tt1650062', 'tt11658994', 'tt10705384', 'tt13051364', 'tt10220164', 'tt10698680', 'tt12532368', 'tt15881484', 'tt11286314', 'tt10447544', 'tt0969332', 'tt10007352', 'tt11296192', 'tt1135084', 'tt1099209', 'tt1082075', 'tt10918500', 'tt11503750', 'tt1

In [41]:
dic_peli={'id_peli':[], 'actores_10':[]}
for id_peli in tqdm(lista_id):
    
    tupla_una_peli=tuplas_actores(id_peli)
    print(tupla_una_peli)
    dic_peli['actores_10'].append(tupla_una_peli)
    dic_peli['id_peli'].append(id_peli)
    
print(dic_peli)

  0%|          | 0/9037 [00:00<?, ?it/s]

  0%|          | 1/9037 [07:50<1181:10:20, 470.59s/it]

[('Sam Buchanan', 'NULL', ['Reparto', 'Reparto'], ['su papel en This Is Martin Bonner', 'Shameless', 'Mad Men'], 'NULL'), ('William French', 'NULL', ['Producción', 'Reparto', 'Dirección de la segunda unidad o asistencia a dirección'], 'NULL', 'NULL'), ('Colin Morgan', 'NULL', ['Reparto', 'Casting', 'Reparto'], 'NULL', 'NULL'), ('Jordan Rountree', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Rogue', 'Fatal Crossing', 'His Mother'], 'NULL'), ('Ava Settel', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Affair', "The Vampire's Widow y La nuit des voyous"], 'NULL')]


  0%|          | 1/9037 [08:13<1239:22:46, 493.78s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.158)
Stacktrace:
	GetHandleVerifier [0x00007FF6D9611562+60802]
	(No symbol) [0x00007FF6D958AC62]
	(No symbol) [0x00007FF6D9447CE4]
	(No symbol) [0x00007FF6D941DFDF]
	(No symbol) [0x00007FF6D94C1E57]
	(No symbol) [0x00007FF6D94D98D1]
	(No symbol) [0x00007FF6D94BA923]
	(No symbol) [0x00007FF6D9488FEC]
	(No symbol) [0x00007FF6D9489C21]
	GetHandleVerifier [0x00007FF6D99141FD+3217949]
	GetHandleVerifier [0x00007FF6D9956197+3488183]
	GetHandleVerifier [0x00007FF6D994F11F+3459391]
	GetHandleVerifier [0x00007FF6D96CB926+823622]
	(No symbol) [0x00007FF6D9595FFF]
	(No symbol) [0x00007FF6D9590F24]
	(No symbol) [0x00007FF6D95910B2]
	(No symbol) [0x00007FF6D9581904]
	BaseThreadInitThunk [0x00007FFB9E2D257D+29]
	RtlUserThreadStart [0x00007FFBA05EAA48+40]
